In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk

from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import TruncatedSVD

In [2]:
## importing the data
titles = [line.rstrip() for line in open("/Users/snigdhacheekoty/Documents/GitHub/Latent_Semantic_Analysis/all_books_titles.rtf")]
stopwords = set(word.rstrip() for word in open("/Users/snigdhacheekoty/Documents/GitHub/SentimentAnalysis/stopwords.rtf"))

In [3]:
# add more stopwords specific to this problem
stopwords = stopwords.union({
    'introduction', 'edition', 'series', 'application',
    'approach', 'card', 'access', 'package', 'plus', 'etext',
    'brief', 'vol', 'fundamental', 'guide', 'essential', 'printed',
    'third', 'second', 'fourth', })

In [4]:
## Preprocessing

def my_tokenizer(s):
    s = s.lower() # downcase
    tokens = nltk.tokenize.word_tokenize(s) # split string into words (tokens)
    tokens = [t for t in tokens if len(t) > 2] # remove short words, they're probably not useful
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] # put words into base form
    tokens = [t for t in tokens if t not in stopwords] # remove stopwords
    tokens = [t for t in tokens if not any(c.isdigit() for c in t)] # remove any digits, i.e. "3rd edition"
    return tokens

word_index_map = {}
current_index = []
all_tokens = []
all_titles = []
index_word_map = []
for title in titles:
    # for the titles that have invalid ASCII
    try:
        title = title.encode("ASCII", "ignore") # If the title has invalid ASCII, we try to encode it in ASCII,
        #... if it doesn't happen, we are going to ignore it
        all_titles = all_titles.append(title)
        tokens = my_tokenizer(title)
        all_tokens = all_tokens.append(tokens)
        for token in tokens:
            if token not in word_index_map:
                word_index_map[token] = current_index
                current_index += 1
                index_word_map.append(token)
    except:
        pass  

In [6]:
# now let's create our input matrices - just indicator variables for this example - works better than proportions
def tokens_to_vector(tokens):
    x = np.zeros(len(word_index_map))
    for t in tokens:
        i = word_index_map[t]
        x[i] = 1
    return x



In [8]:
N = len(all_tokens)
D = len(word_index_map)
X = np.zeros((D, N)) # terms will go along rows, documents along columns
i = 0
for tokens in all_tokens:
    X[:,i] = tokens_to_vector(tokens)
    i += 1



In [15]:
def main():
    svd = TruncatedSVD()
    Z = svd.fit_transform(X)
    plt.scatter(Z[:,0], Z[:,1])
    for i in xrange(D):
        plt.annotate(s=index_word_map[i], xy=(Z[i,0], Z[i,1]))
    plt.show()
